In [79]:
import pandas as pd  # Importación de pandas para manipulación de datos
import re  # Importación de expresiones regulares para extracción de números
import numpy as np  # Importación de numpy para manejo de arrays numéricos
from tqdm import tqdm  # Importación de tqdm para barras de progreso
from sklearn.metrics.pairwise import cosine_similarity  # Importación de la función para calcular la similitud coseno
from transformers import AutoTokenizer, AutoModel  # Importación de herramientas de Hugging Face para modelos de lenguaje
import torch  # Importación de PyTorch para el procesamiento de modelos de aprendizaje automático
import sys
import sys
import os
sys.path.append(os.path.abspath('../'))
from data_utils.manipulacion_de_datos.maniuplacion_de_datos import estandarizado_columnas
                                                                  

# Pre procesamiento 


In [80]:
# Cargar modelo preentrenado
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
model = AutoModel.from_pretrained('dccuchile/bert-base-spanish-wwm-cased')

# Función para obtener embeddings
def obtener_embedding(texto, tokenizer, model):
    tokens = tokenizer(texto, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        output = model(**tokens)
    return output.last_hidden_state.mean(dim=1).numpy()

# Cargar datos
df_master = pd.read_csv('df_MasterProductos_Filtro.csv')
df_proveedor = pd.read_csv('ABBVIE.csv', )
df_proveedor = estandarizado_columnas(df_proveedor,'descripcion', 'presentacion', 'descripcion_limpia_proveedor')

Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [81]:
# Obtener embeddings de los productos master
df_master['embedding_Master'] = df_master['descripcion_limpia_producto'].apply(
    lambda x: obtener_embedding(x, tokenizer, model)
)

# Obtener embeddings de los productos del proveedor
df_proveedor['embedding_Proveedor'] = df_proveedor['descripcion_limpia_proveedor'].apply(
    lambda x: obtener_embedding(x, tokenizer, model)
)

# Modelo 

In [82]:
df_master['embedding_Master'] 

0     [[-0.23708273, 0.43097925, -0.6396462, 0.34665...
1     [[-0.15039249, 0.3770446, -0.626851, 0.3519937...
2     [[-0.05165407, 0.08066824, -0.66579986, 0.3826...
3     [[0.025983853, 0.0017536337, -0.5659652, 0.415...
4     [[-0.17178698, 0.11327641, -0.6755684, 0.44862...
5     [[0.023351634, 0.1531327, -0.46274215, 0.38065...
6     [[0.13862811, 0.03878904, -0.40951735, 0.09430...
7     [[0.2214301, 0.07301196, -0.5797721, 0.2236536...
8     [[-0.0680114, -0.067341626, -0.3875184, 0.0813...
9     [[-0.024668997, -0.09351689, -0.3786446, 0.009...
10    [[0.23030677, -0.053384535, -0.46321782, 0.174...
11    [[-0.003784079, 0.02038069, 0.13646379, -0.252...
12    [[0.0158983, -0.09849517, -0.27595076, 0.11002...
13    [[0.094218716, -0.11914583, -0.16052884, 0.148...
14    [[0.09833872, -0.10977365, -0.20405895, 0.1557...
15    [[-0.46610996, 0.39341718, 0.06529043, -0.4739...
16    [[0.39241207, 0.20349364, -0.60381615, -0.0267...
17    [[0.36723667, 0.21821222, -0.6421874, 0.05

In [83]:
df_proveedor['embedding_Proveedor'] 

0     [[-0.00036333158, -0.121306695, -0.02509405, -...
1     [[0.039201435, -0.06423703, -0.18669297, -0.09...
2     [[-0.08804746, -0.14528003, -0.18327762, -0.15...
3     [[0.015499055, 0.30273744, -0.17271008, -0.440...
4     [[0.13719383, 0.2957174, -0.18936808, -0.41752...
5     [[0.07378752, 0.23473603, -0.2533665, -0.39393...
6     [[-0.11794027, -0.14087155, -0.3921879, 0.0219...
7     [[-0.3438007, 0.2738461, -0.46558484, -0.22402...
8     [[-0.337425, 0.024610186, -0.2615152, -0.20678...
9     [[-0.073238, -0.009511469, -0.20886673, -0.460...
10    [[-0.49299404, -0.06256475, 0.16610293, -0.289...
11    [[-0.110288054, 0.12482095, -0.08721691, -0.19...
12    [[-0.12872155, -0.12837894, -0.017920848, -0.2...
13    [[-0.24433814, -0.19635265, -0.077770285, -0.2...
14    [[0.07271305, 0.12276241, 0.03909163, -0.47566...
15    [[-0.26039615, -0.12892337, 0.07866887, -0.497...
16    [[-0.22786364, -0.2708873, 0.40160605, -0.5473...
17    [[0.03719409, -0.18427302, 0.07510432, -0.

In [84]:
# Comparar similitudes
similitudes = []

for i, row_proveedor in df_proveedor.iterrows():
    emb_proveedor = row_proveedor['embedding_Proveedor']
    mejores_matches = []
    
    for j, row_master in df_master.iterrows():
        emb_master = row_master['embedding_Master']
        
        # Calcular similitud coseno
        similitud = cosine_similarity(emb_proveedor.reshape(1, -1), emb_master.reshape(1, -1))[0][0]
        
        # Guardar resultados
        mejores_matches.append((row_master['descripcion_limpia_producto'], similitud))

    # Ordenar por mayor similitud y obtener los 3 mejores matches
    mejores_matches.sort(key=lambda x: x[1], reverse=True)
    mejores_matches = mejores_matches[:3]  # Tomar solo los 3 primeros

    # Agregar los mejores matches
    for mejor_match in mejores_matches:
        similitudes.append({
            'descripcion_proveedor': row_proveedor['descripcion_limpia_proveedor'],
            'mejor_match_master': mejor_match[0],
            'similaridad': mejor_match[1]
        })

# Convertir resultados en DataFrame
df_resultado = pd.DataFrame(similitudes)


In [85]:
df_resultado

,descripcion_proveedor,mejor_match_master,similaridad
0,"HUMIRA AC x 2 (20mg) 2 JERINGAS PRELL. X 0,2 ml",skyrizi 75 miligramos jeringa prellenada 2,0.903576
1,"HUMIRA AC x 2 (20mg) 2 JERINGAS PRELL. X 0,2 ml",humira jeringa prellenada 2 08 mililitros,0.899604
2,"HUMIRA AC x 2 (20mg) 2 JERINGAS PRELL. X 0,2 ml",skyrizi 150 miligramos lapicera prellenada 1,0.895096
3,"HUMIRA AC x 2 (40mg) 2 LAPICERAS PRELL. X 0,4 ml",humira jeringa prellenada 2 08 mililitros,0.910324
4,"HUMIRA AC x 2 (40mg) 2 LAPICERAS PRELL. X 0,4 ml",skyrizi 75 miligramos jeringa prellenada 2,0.906883
...,...,...,...
58,BOTOX Vial 200 U,humira pediatrico vial 2 08 mililitros,0.885081
59,BOTOX Vial 200 U,zemplar 2 microgramo capsulas 30,0.884230
60,OZURDEX 0.7 MG C/1,venclexta kit de inicio 1,0.876740
61,OZURDEX 0.7 MG C/1,viekira pak 28 blis 4 comprimidos cu,0.875662


In [90]:
# Comparar similitudes
similitudes = []

for i, row_proveedor in df_proveedor.iterrows():
    emb_proveedor = row_proveedor['embedding_Proveedor']
    mejores_matches = []
    
    for j, row_master in df_master.iterrows():
        emb_master = row_master['embedding_Master']
        
        # Calcular similitud coseno
        similitud = cosine_similarity(emb_proveedor.reshape(1, -1), emb_master.reshape(1, -1))[0][0]
        
        # Guardar resultados
        mejores_matches.append((row_master['descripcion_limpia_producto'], similitud, row_master['niprod']))

    # Ordenar por mayor similitud y obtener los 3 mejores matches
    mejores_matches.sort(key=lambda x: x[1], reverse=True)
    mejores_matches = mejores_matches[:3]  # Tomar solo los 3 primeros

    # Agregar los mejores matches
    for mejor_match in mejores_matches:
        similitudes.append({
            'descripcion_limpia_proveedor': row_proveedor['descripcion_limpia_proveedor'],
            'descripcion_match_master': mejor_match[0],
            'niprod': mejor_match[2],  # Agregar niprod como llave
            'similitud': mejor_match[1]
        })

# Convertir resultados en DataFrame
df_resultado = pd.DataFrame(similitudes)

# Ahora, vamos a hacer el merge con el df_proveedor para agregar la columna niprod
df_proveedor_resultado = pd.merge(df_proveedor, df_resultado[['descripcion_limpia_proveedor', 'niprod']], 
                                  on='descripcion_limpia_proveedor', how='left')

In [91]:
df_proveedor_resultado

,condición de precios,cod_producto_proveedor,descripcion,unnamed: 3,unnamed: 4,presentacion,unidades,unnamed: 7,precio s/iva,unnamed: 9,descuento,unnamed: 11,total,cod_barra,descripcion_limpia_proveedor,embedding_Proveedor,niprod
0,NaN,20015888,HUMIRA AC x 2 (20mg),NaN,NaN,"2 JERINGAS PRELL. X 0,2 ml",NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,"HUMIRA AC x 2 (20mg) 2 JERINGAS PRELL. X 0,2 ml","[[-0.00036333158, -0.121306695, -0.02509405, -...",119565.0
1,NaN,20015888,HUMIRA AC x 2 (20mg),NaN,NaN,"2 JERINGAS PRELL. X 0,2 ml",NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,"HUMIRA AC x 2 (20mg) 2 JERINGAS PRELL. X 0,2 ml","[[-0.00036333158, -0.121306695, -0.02509405, -...",119549.0
2,NaN,20015888,HUMIRA AC x 2 (20mg),NaN,NaN,"2 JERINGAS PRELL. X 0,2 ml",NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,"HUMIRA AC x 2 (20mg) 2 JERINGAS PRELL. X 0,2 ml","[[-0.00036333158, -0.121306695, -0.02509405, -...",124898.0
3,NaN,20005289,HUMIRA AC x 2 (40mg),NaN,NaN,"2 LAPICERAS PRELL. X 0,4 ml",NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,"HUMIRA AC x 2 (40mg) 2 LAPICERAS PRELL. X 0,4 ml","[[0.039201435, -0.06423703, -0.18669297, -0.09...",119549.0
4,NaN,20005289,HUMIRA AC x 2 (40mg),NaN,NaN,"2 LAPICERAS PRELL. X 0,4 ml",NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,"HUMIRA AC x 2 (40mg) 2 LAPICERAS PRELL. X 0,4 ml","[[0.039201435, -0.06423703, -0.18669297, -0.09...",119565.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,NaN,93251AR,BOTOX,NaN,NaN,Vial 200 U,NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,BOTOX Vial 200 U,"[[-0.06815099, -0.09752112, -0.2971691, -0.429...",119553.0
59,NaN,93251AR,BOTOX,NaN,NaN,Vial 200 U,NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,BOTOX Vial 200 U,"[[-0.06815099, -0.09752112, -0.2971691, -0.429...",119577.0
60,NaN,93348AR,OZURDEX,NaN,NaN,0.7 MG C/1,NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,OZURDEX 0.7 MG C/1,"[[-0.39663455, 0.35714456, -0.051247146, -0.23...",119575.0
61,NaN,93348AR,OZURDEX,NaN,NaN,0.7 MG C/1,NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,OZURDEX 0.7 MG C/1,"[[-0.39663455, 0.35714456, -0.051247146, -0.23...",119576.0


In [88]:
# Comparar similitudes y agregar resultados
similitudes = []

for i, row_proveedor in df_proveedor.iterrows():
    emb_proveedor = row_proveedor['embedding_Proveedor']
    mejores_matches = []
    
    for j, row_master in df_master.iterrows():
        emb_master = row_master['embedding_Master']
        
        # Calcular similitud coseno
        similitud = cosine_similarity(emb_proveedor.reshape(1, -1), emb_master.reshape(1, -1))[0][0]
        
        # Guardar resultados, añadiendo niprod
        mejores_matches.append((row_master['descripcion_limpia_producto'], similitud, row_master['niprod']))

    # Ordenar por mayor similitud y obtener los 3 mejores matches
    mejores_matches.sort(key=lambda x: x[1], reverse=True)
    mejores_matches = mejores_matches[:3]  # Tomar solo los 3 primeros

    # Agregar los mejores matches
    for mejor_match in mejores_matches:
        similitudes.append({
            'descripcion_limpia_proveedor': row_proveedor['descripcion_limpia_proveedor'],
            'descripcion_match_master': mejor_match[0],
            'niprod': mejor_match[2],  # Agregar niprod como llave
            'similitud': mejor_match[1]
        })

# Convertir resultados en DataFrame
df_resultado = pd.DataFrame(similitudes)

# Ahora, vamos a hacer el merge con el df_proveedor para agregar la columna niprod
df_proveedor_resultado = pd.merge(df_proveedor, df_resultado[['descripcion_limpia_proveedor', 'niprod']], 
                                  on='descripcion_limpia_proveedor', how='left')

# Hacer otro merge para agregar la columna 'descripcion_limpia_producto' del master
df_proveedor_resultado = pd.merge(df_proveedor_resultado, df_master[['niprod', 'descripcion_limpia_producto']], 
                                  on='niprod', how='left')

# Ahora, el DataFrame df_proveedor_resultado tiene la columna 'descripcion_limpia_producto' y 'niprod'
# Puedes visualizarlo



In [89]:
# Mostrar el DataFrame final
df_proveedor_resultado


,condición de precios,cod_producto_proveedor,descripcion,unnamed: 3,unnamed: 4,presentacion,unidades,unnamed: 7,precio s/iva,unnamed: 9,descuento,unnamed: 11,total,cod_barra,descripcion_limpia_proveedor,embedding_Proveedor,niprod,descripcion_limpia_producto
0,NaN,20015888,HUMIRA AC x 2 (20mg),NaN,NaN,"2 JERINGAS PRELL. X 0,2 ml",NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,"HUMIRA AC x 2 (20mg) 2 JERINGAS PRELL. X 0,2 ml","[[-0.00036333158, -0.121306695, -0.02509405, -...",119565.0,skyrizi 75 miligramos jeringa prellenada 2
1,NaN,20015888,HUMIRA AC x 2 (20mg),NaN,NaN,"2 JERINGAS PRELL. X 0,2 ml",NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,"HUMIRA AC x 2 (20mg) 2 JERINGAS PRELL. X 0,2 ml","[[-0.00036333158, -0.121306695, -0.02509405, -...",119549.0,humira jeringa prellenada 2 08 mililitros
2,NaN,20015888,HUMIRA AC x 2 (20mg),NaN,NaN,"2 JERINGAS PRELL. X 0,2 ml",NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,"HUMIRA AC x 2 (20mg) 2 JERINGAS PRELL. X 0,2 ml","[[-0.00036333158, -0.121306695, -0.02509405, -...",124898.0,skyrizi 150 miligramos lapicera prellenada 1
3,NaN,20005289,HUMIRA AC x 2 (40mg),NaN,NaN,"2 LAPICERAS PRELL. X 0,4 ml",NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,"HUMIRA AC x 2 (40mg) 2 LAPICERAS PRELL. X 0,4 ml","[[0.039201435, -0.06423703, -0.18669297, -0.09...",119549.0,humira jeringa prellenada 2 08 mililitros
4,NaN,20005289,HUMIRA AC x 2 (40mg),NaN,NaN,"2 LAPICERAS PRELL. X 0,4 ml",NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,"HUMIRA AC x 2 (40mg) 2 LAPICERAS PRELL. X 0,4 ml","[[0.039201435, -0.06423703, -0.18669297, -0.09...",119565.0,skyrizi 75 miligramos jeringa prellenada 2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,NaN,93251AR,BOTOX,NaN,NaN,Vial 200 U,NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,BOTOX Vial 200 U,"[[-0.06815099, -0.09752112, -0.2971691, -0.429...",119553.0,humira pediatrico vial 2 08 mililitros
59,NaN,93251AR,BOTOX,NaN,NaN,Vial 200 U,NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,BOTOX Vial 200 U,"[[-0.06815099, -0.09752112, -0.2971691, -0.429...",119577.0,zemplar 2 microgramo capsulas 30
60,NaN,93348AR,OZURDEX,NaN,NaN,0.7 MG C/1,NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,OZURDEX 0.7 MG C/1,"[[-0.39663455, 0.35714456, -0.051247146, -0.23...",119575.0,venclexta kit de inicio 1
61,NaN,93348AR,OZURDEX,NaN,NaN,0.7 MG C/1,NaN,NaN,NaN,NaN,NaN,NaN,-,NaN,OZURDEX 0.7 MG C/1,"[[-0.39663455, 0.35714456, -0.051247146, -0.23...",119576.0,viekira pak 28 blis 4 comprimidos cu
